# Training a Federated Model 

In [ ]:
import os 
os.chdir('../')

In [32]:
import os 
import numpy as np 
import pandas as pd 
import torch 
import torch.nn as nn 
import torch.optim as optim 
import torch.nn.functional as F 
import gc
from torch.utils.data import Dataset, DataLoader
import tqdm
import sklearn.metrics as metrics
from common.dataset import *
from common.models import * 
from common.utils import * 
import logging 
import argparse
import datetime as dt 
import random 
import copy

In [72]:
# len(keys)*len(weights) - can get expensive
def average_weights(w):
    w_avg = copy.deepcopy(w[0])
    for k in w_avg.keys():
        for i in range(1, len(w)):
            w_avg[k] += w[i][k]
        w_avg[k] = torch.div(w_avg[k], len(w))
    return w_avg

In [6]:
DATADIR = 'fl_data\HHAR'

In [7]:
FL_AGENTS = os.listdir(DATADIR)
FL_AGENTS

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i']

In [19]:
FL_SAMPLE = 0.4

In [30]:
global_model = FFN(48, 10)

In [74]:
for each_round in range(1): 
    agents_to_train = random.choices(FL_AGENTS, k= int(FL_SAMPLE * len(FL_AGENTS)))
    model_list = []
    for each_agent in agents_to_train: 
        # read the data. 
        test, train = [pd.read_csv(os.path.join(DATADIR, each_agent, i)) for i in os.listdir(os.path.join(DATADIR, each_agent))]
        train = train.fillna(0)
        test = test.fillna(0)
        trainData, testData = HARData(train), HARData(test)
        trainLoader, testLoader = getDataLoader(trainData, testData)
        loss = nn.CrossEntropyLoss()
        model = copy.deepcopy(global_model)
        optimizer = optim.Adam(model.parameters())
        model.train()
        for epoch in range(10): 
            # train each epoch. 
            for i, (x, y) in tqdm.tqdm(enumerate(trainLoader), total =len(trainLoader)): 
                yhat = model(x)
                batch_loss = loss(yhat, y)
                optimizer.zero_grad()
                batch_loss.backward()
                optimizer.step()
            # for i, (x, y) in enumerate(testLoader): 
        model_list.append(model.state_dict())
    # average weight at end of round. 
    avg_weights = average_weights(model_list)
    global_model.load_state_dict(avg_weights)
    # run tests on every agent's test data. 
    for each_agent in FL_AGENTS: 
        test, train = [pd.read_csv(os.path.join(DATADIR, each_agent, i)) for i in os.listdir(os.path.join(DATADIR, each_agent))]
        train = train.fillna(0)
        test = test.fillna(0)
        trainData, testData = HARData(train), HARData(test)
        trainLoader, testLoader = getDataLoader(trainData, testData)
        print('Round: {0}, Agent: {1}'.format(each_round, each_agent))
        print(get_accuracy(global_model, testLoader))


100%|██████████| 6/6 [00:05<00:00,  1.20it/s]
Round: 0, Agent: a
[0.345703125, 0.3290203327171904]
Round: 0, Agent: b
[0.767578125, 0.77001953125, 0.7058823529411765]
Round: 0, Agent: c
[0.5029296875, 0.5112293144208038]
Round: 0, Agent: d
[0.6787109375, 0.6641221374045801]
Round: 0, Agent: e
[0.56201171875, 0.5689834024896265]
Round: 0, Agent: f
[0.7001953125, 0.6872800402212167]
Round: 0, Agent: g
[0.62982689747004]
Round: 0, Agent: h
[0.44293614881850174]
Round: 0, Agent: i
[0.2303370786516854]
